# Imports and functions


## Jeff notes

MLE(dfa, corpus) = dfa' 

create_SL2_dfa (alphabet

getalphabet(corpus) = set_of_letters 

Q = all strings of length less than k

I = "" 

F = Q

delta(q,a) = k-1 suffix of qa 

example: (a,b) = k-1 suffix of ab which is b (k=2)

Example; Suppose k=4. Then delta(aaa,b) = 3 suffix of aaab, which is aab 

Also if qa is of length less than k-1 than the k-1 suffix of qa is just qa 

3 suffix of aa is aa

BIGRAM: all possible states given alphabet (given k)

42 SP machines, 1 SL machine

In [2]:
from pyfoma import FST, State

fish = "⋉"

def data_iterator(path):
    """Yield a word as a list of tokens"""
    with open (path, "r") as fin:
        for line in fin.readlines():
            yield line.split()
            
def make_alphabet(path_to_corpus):
    alph = set()
    for word in data_iterator(path_to_corpus):
        alph = alph.union(set(word))
    return alph

def make_2_SL_dfa(alphabet):
    pass
    




make_alphabet("data/LearningData.txt")

{'N',
 'a',
 'b',
 'bh',
 'bv',
 'ch',
 'd',
 'dh',
 'dy',
 'dz',
 'dzv',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'm',
 'mh',
 'n',
 'nh',
 'ny',
 'o',
 'p',
 'pf',
 'r',
 's',
 'sh',
 'sv',
 't',
 'ts',
 'tsv',
 'ty',
 'u',
 'v',
 'vh',
 'w',
 'y',
 'z',
 'zh',
 'zv'}

1